In [1]:
import pandas as pd
import numpy as np
import pickle

In [12]:
dataset = 'ppi_sc_st_4'
######## define label
root= '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/data/'
df = pd.read_csv(root+dataset+'.csv')
disease_root = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/thesis/HBDM/data/disease/'
disease_source = 'DIS_CAD'
diseasedf = pd.read_csv(disease_root+disease_source+'.tsv',sep='\t')
pos_genes = diseasedf['Gene'].to_list()

In [4]:
df.columns

Index(['ppi_1d', 'ppi_2d', 'ppi_3d', 'ppi_4d', 'ppi_re', 'gene', 'sc_1d',
       'sc_2d', 'sc_3d', 'sc_4d', 'sc_re', 'st_1d', 'st_2d', 'st_3d', 'st_4d',
       'st_re'],
      dtype='object')

In [13]:
df = df[['ppi_1d', 'ppi_2d', 'ppi_3d', 'ppi_4d', 'gene']]

In [7]:
from sklearn.neighbors import KDTree
from sklearn.model_selection import KFold

In [10]:
local_stringdb = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/data/string/lfs-stringdb/'
# load local STRING database and names
df = pd.read_csv(local_stringdb+'9606.protein.info.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
df['preferred_name'] = df['preferred_name'].str.upper()
stringId2name = df.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = df.set_index('preferred_name')['#string_protein_id'].to_dict()
df = pd.read_csv(local_stringdb+'9606.protein.aliases.v12.0.txt', sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
df['alias'] = df['alias'].str.upper()
aliases2stringId = df.set_index('alias')['#string_protein_id'].to_dict()

string_score_transform = lambda x: -np.log(x/1000)

network = pd.read_csv(local_stringdb+'9606.protein.physical.links.detailed.v12.0.txt', sep=' ', header=0).convert_dtypes().replace(0, float('nan'))
network['combined_score'] = network['combined_score'].apply(string_score_transform)

def convert_stringId(alias):
    try:
        stringId = name2stringId[alias]
    except:
        #print(alias, 'can\'t be converted by name2stringId! Now trying aliases2stringId.')
        try:
            stringId = aliases2stringId[alias]
        except:
            #print(alias, 'can\'t be converted by aliases2stringId! Now return None.')
            stringId = None
    #print(alias, stringId)
    return stringId

def read_string_net_from_df(df, weight_choose='combined_score'): # default combined score
    string_net = {}
    for row in df.itertuples(index=False):
        start_node, end_node, weight = row.protein1, row.protein2, getattr(row, weight_choose)
        if start_node not in string_net:
            string_net[start_node] = {}
        string_net[start_node][end_node] = weight
        if end_node not in string_net:
            string_net[end_node] = {}
        string_net[end_node][start_node] = weight
    return string_net

In [5]:
import networkx as nx
import itertools
from sklearn.model_selection import KFold

In [6]:
G = nx.from_pandas_edgelist(network, source='protein1', target='protein2', edge_attr='combined_score', create_using=nx.Graph)

In [7]:
len(G.edges)

738805

In [19]:
disease_source = 'OT_CAD'
diseasedf = pd.read_csv(disease_root+disease_source+'.tsv',sep='\t')
pos_genes = diseasedf['symbol']

In [22]:
disease_source = 'cad_literature'
diseasedf = pd.read_csv(disease_root+disease_source+'.csv')
pos_genes = diseasedf['gene']

In [25]:
genticpath = '/novo/omdb/pds02/PDS2843/data/sprint_tid_ascvd/gzn/310_targets/CAD-Genetic-Target-Discovery-2023_010323.xlsx'
geneticdf=pd.read_excel(genticpath,sheet_name='MASTER')
pos_genes = geneticdf['Gene']

In [14]:
pos_genesid = list(set(list(map(convert_stringId,pos_genes))))
pos_genesid = list(set(pos_genesid)&set(G.nodes))
len(pos_genesid)

NameError: name 'G' is not defined

In [16]:
def index_dict(df):
    return dict(zip(df['gene'], df.index))
df['value'] =df.index
dfdict = index_dict(df[['value','gene']])

In [23]:
list(map(dfdict,pos_genes))

TypeError: 'dict' object is not callable

In [20]:
pos_genesid = list(set(list(map(dfdict,pos_genes))))
pos_genesid = list(set(pos_genesid)&set(dfdict.values()))
len(pos_genesid)

TypeError: get expected at least 1 argument, got 0

In [ ]:
group_node = pos_genesid
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for k in k_values:
    k+=1
    pre = []
    cov = []
    neibor_dict = dict()
    # Build a k-d tree from the points
    kdtree = KDTree(df[[col for col in df.columns if col.endswith('d')]].to_numpy(), leaf_size=20)
    for train_index, test_index in kf.split(group_node):
        train_nodes = [group_node[i] for i in train_index]
        test_nodes = [group_node[i] for i in test_index]
        start = []
        dist = []
        neighbor = []

        for i in train_nodes:
            given_point = df[df['node']==i][[col for col in df.columns if col.endswith('d')]].to_numpy()
            # Perform a k-NN search to find the k+1 nearest neighbors
            distances, indices = kdtree.query(given_point, k=k)
            start += (k-1)*[i]
            dist += distances.reshape(-1).tolist()[1:]
            neighbor += indices.reshape(-1).tolist()[1:]
        # random_nodes = random.sample(list(set(df['node'].tolist())-set(train_nodes)), (k-1))
        for gene in neighbor:
            if gene in neibor_dict:
                neibor_dict[gene] += 1
            else:
                neibor_dict[gene] = 1
        tp_dict = {key: neibor_dict[key] for key in neibor_dict.keys() if key in test_nodes}
        pre.append(sum(tp_dict.values())/sum(neibor_dict.values()))                
        cov.append(len(tp_dict.keys())/len(test_nodes))
    print(sum(pre) / len(pre),sum(cov) / len(cov))
        

In [40]:
for k in [3,4,5,6,7,8,9,10]:
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    pre = []
    cov = []
    for train_index, test_index in kf.split(pos_genesid):
        train = [pos_genesid[i] for i in train_index]
        test = [pos_genesid[i] for i in test_index]
        neibors = dict()
        for start in train:
            distances = nx.shortest_path_length(G, start, weight='weight')
            distances = {node: distance for node, distance in distances.items()}
            distances = dict(sorted(distances.items(), key=lambda item: item[1]))
            nearest_nodes = list(distances.keys())[1:k+1]
            for gene in nearest_nodes:
                if gene in neibors:
                    neibors[gene] += 1
                else:
                    neibors[gene] = 1
        tp_dict = {key: neibors[key] for key in neibors.keys() if key in test}
        pre.append(sum(tp_dict.values())/sum(neibors.values()))                
        cov.append(len(tp_dict.keys())/len(test))
    print(sum(pre) / len(pre),sum(cov) / len(cov))


KeyboardInterrupt: 

In [43]:
for threshold in [3,4,5,6,7,8,9,10]:
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    pre = []
    cov = []
    for train_index, test_index in kf.split(pos_genesid):
        train = [pos_genesid[i] for i in train_index]
        test = [pos_genesid[i] for i in test_index]
        pair_combinations = list(itertools.combinations(train, 2))
        neibors = dict()
        for pairs in pair_combinations:
            path = nx.shortest_path(G, pairs[0], pairs[1],weight='weight')
            if len(path) > threshold:
                continue
            else:
                for gene in path[1:-1]:
                    if gene in neibors:
                        neibors[gene] += 1
                    else:
                        neibors[gene] = 1
        tp_dict = {key: neibors[key] for key in neibors.keys() if key in test}
        pre.append(sum(tp_dict.values())/sum(neibors.values()))                
        cov.append(len(tp_dict.keys())/len(test))
    print(sum(pre) / len(pre),sum(cov) / len(cov))

KeyboardInterrupt: 

In [ ]:
dis cad 1532
0.06977141547512938 0.7428221668689191
0.05936421325654512 0.8139745800600371
0.05901639688122533 0.8146281748312789
0.05910539875594115 0.8146281748312789
0.05911384610086836 0.8146281748312789
0.05911393544791371 0.8146281748312789
0.05911393544791371 0.8146281748312789
0.05911393544791371 0.8146281748312789

ot cad 262
0.01312909619495666 0.31320754716981136
0.01383285127990913 0.5613207547169812
0.013966368130537407 0.5613207547169812
0.013966368130537407 0.5613207547169812
0.013966368130537407 0.5613207547169812
0.013966368130537407 0.5613207547169812
0.013966368130537407 0.5613207547169812
0.013966368130537407 0.5613207547169812

cad liter 149
0.007672976576734584 0.1413793103448276
0.005059493412264257 0.2418390804597701
0.005089489534095742 0.2418390804597701
0.005087827844910528 0.2418390804597701
0.005087827844910528 0.2418390804597701
0.005087827844910528 0.2418390804597701
0.005087827844910528 0.2418390804597701
0.005087827844910528 0.2418390804597701

nn cad 366
0.004609683299488819 0.19403924472417625
0.004195223440859282 0.29777860051832655
0.004149633937581671 0.3059977786005183
0.004147331500966463 0.3059977786005183
0.004147331500966463 0.3059977786005183
0.004147331500966463 0.3059977786005183
0.004147331500966463 0.3059977786005183
0.004147331500966463 0.3059977786005183

In [17]:
with open(r'D:\study\thesis\project\HBDM-main\data\complexes\complexes.pkl', 'rb') as f:
    complexs = pickle.load(f)
protein_names = list(aliases2stringId.keys())
protein_names.extend(list(name2stringId.keys()))

humans = set(value_to_index_mapping.keys())

complexs_id = dict()

for complex_name in complexs:
    # folder_path = 'D:/study/thesis/project/HBDM-main/ppi_results/test_results/'+complex_name
    # os.mkdir(folder_path)
    group_node = []
    for gene in complexs[complex_name]:
        if gene in protein_names:
            stringid = convert_stringId(gene)
            # stringid = int(stringid[9:])
            # if stringid in humans:
            #     node = value_to_index_mapping[stringid]
            group_node.append(stringid)
    if len(set(group_node)) != 1:
        complexs_id[complex_name] = group_node
    else:
        print(complex_name)


In [18]:
col1 = []
precision_col = []
coverage_col = []
for complex_name in complexs_id:
    col1.append(complex_name)
    points = complexs_id[complex_name]
    for top in [20]:
        precision = []
        coverage = []
        for start_gene in points:
            test_nodes = list(set(points)-set(start_gene))
            true_pre = []
            subdf = network[network['protein1']==start_gene]
            ranked = subdf.sort_values(by='combined_score')
            predicted=ranked[:top]['protein2'].tolist()
            for i in predicted:
                if i in test_nodes:
                    true_pre.append(i)
            precision.append(len(true_pre)/len(predicted))
            coverage.append(len(set(true_pre))/len(test_nodes))
        # print('top-',top,'\t',complex_name,' precision: ', sum(precision)/len(precision))
        # print(complex_name,' coverage: ', sum(coverage)/len(coverage))
        precision_col.append(sum(precision)/len(precision))
        coverage_col.append(sum(coverage)/len(coverage))

In [19]:
len(col1)

173

In [20]:
name = 'sp_detailed_top20'
result = pd.DataFrame({'complex': col1, 'precision_20':precision_col,'coverage_20':coverage_col})
results_name = name +'.csv'
result.to_csv('D:/study/thesis/project/HBDM-main/ppi_results/test_results/complexes/'+results_name,index=False)